<a href="https://colab.research.google.com/github/Codeybab/COS324_HW/blob/main/PA6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 6: Value Iteration

## Problem Setting

**Gridworld**: In this assignment, we use the popular RL toolkit  [OpenAI Gym](https://gym.openai.com/)  to implement _value iteration_ on a gridworld environment (Lecture 21). In the gridworld environment we consider, an agent needs to navigate on a 2-D plane by using 4 actions (left, right, up, and down). It starts at a designated state (starting point "S"), and needs to try and reach a goal state ("G"). The agent controls the movement of a character in a grid world. Some tiles on the grid are walkable ("S", "G", and "F"), and others ("H") lead to the agent falling into the water, which ends the episode. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

**Rewards**: The environment is such that reaching the goal state gives the agent a `+1` reward, and it gets a `0` reward in other states (detailed description below). Hence, reaching the goal state is equivalent to maximizing the rewards it can accumulate.

**Uncertainty**: The environment also has some uncertainty. That is, the movement direction of the agent is uncertain and only partially depends on the chosen direction. In our setting (detailed below), each tile in the grid is "slippery", which corresponds to the following uncertainty; if the agent chose to move up/down, it will be able to do so with probability `1/3`, but with probability `2/3` it will move right/left (`1/3` right, `1/3` left). Similarly for the case that the agent chose to move right/left, it might move up/down instead. However, you will always move by 1 tile only (i.e., no "jumps"). 

This is visualized below:

<img src="fig/penguin_walk.png">

In a) the agent (penguin) tries to walk down, in b) it tries to walk right, and c) shows a potential pitfall where just by walking by a hole "H" (and not directly into it), the agent still has a `1/3` chance to fall into the hole, which ends the episode with no reward.


Note that: 
* The specific transition probabilities don't matter much here, but it is important to keep in mind is that the action you choose to move towards only partially determined where you actually move, due to the uncertianty ("slipperiness") of the environment. 

* Recall that while you can see the whole gridworld (all the states), the agent only has information about its current state!

## Environment: Frozen Lake 
Winter is here. You and your friends are tossing around a frisbee at the park when you make a wild throw that leaves the frisbee stranded out in the middle of
a lake. The water is mostly frozen ("F"), but there are a few holes ("H") where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend, and sometimes slip into another, adjacent tile! The surface is described using a grid like the following $8 \times 8$ example:

    SFFFFFFF
    FFFFFFFF
    FFFHFFFF
    FFFFFHFF
    FFFHFFFF
    FHHFFFHF
    FHFFHFHF
    FFFHFFFG


    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located


The episode ends when you reach the goal or fall in a hole "H". You receive a reward of 1 if you reach the goal, and 0 otherwise. The steps that you can make are one of the following: LEFT = 0, DOWN = 1, RIGHT = 2, UP = 3.

## Visualization:
It is easier to understand how this environment behaves if you see it first.  

#### $\star$ We recommend that before implementing anything, you start by first running all cells of this notebook, and scroll down to see how a **totally-random** agent moves around the frozen lake. $\star$ 
 
You will see it fail many times. In what follows, we will implement a much better agent that can navigate to the goal.
      

### Notation

Mapping of the OpenAI gym constructs to the notation used in lectures:
* The value function $V(s)$, is implemented here as a small array, of size corrsponsing to number of states, such that `V[s]` contains the value $V(s)$. 
* The transition probabilities of the environment $P(s'|s,a)$ correspond to the gym object `env.P` described below. 
* In lectures, the reward function $r(a|s, s')$ signified  taking action $a$ while in state $s$ moving to state $s'$. However, in our setting here the reward fixed into the environment is such that $r(a|s=\text{'G'}, s')=1$ and $r(a|s \neq\text{'G'}, s')=0$ for any $a, s'$. In other words, in this setting we only get the reward if we *actually reach* the goal state.

### First, install dependencies (takes ~45 seconds).

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb libav-tools python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
import gym
import numpy as np
import random
from IPython.display import clear_output
import time

In [ ]:
# first, we set some useful parameters: 
gamma = 0.99        # discount factor
theta = 0.000001    # precision of value_iteration
        
# let's set up the frozen lake environment. 
try:
    env = gym.make("FrozenLake8x8-v0")
except:
    env = gym.make("FrozenLake8x8-v1") # v0 might not exist
    
num_states = env.nS   # we have one state per tile in the grid (16 states)
num_actions = env.nA  # we can move up/down/left/right, i.e., 4 actions. 

# NOTE, the transition probabilities we mentioned above (i.e., the "slipperiness"), are already given 
# in the env.P object. Specifically, env.P are the transition probabilities (dictionary dict of dicts of lists)
#           P[s][a] == [(probability, s', reward, done), 
#                       (probability, s', reward, done), 
#                       (probability, s', reward, done), ]

# As an example, for state s=0 (the starting tile "S"),and action a=0 (LEFT), you will have:
#           P[0][0] == [(1/3, 0, 0, False), (1/3, 0, 0, False), (1/3, 4, 0, False)] 
# since:
# with probability 1/3 you'll move UP (but there's no up, so you'll stay put at s=0),
# with probability 1/3 you'll move LEFT (but there's no left, so you'll stay put at s=0),
# with probability 1/3 you'll move DOWN (so you'll get to the tile below, which is s'=4).


We will now implement the Value Iteration algorithm you have seen in class. The pseudo-code is given below: 

1. Parameter: a small threshold $\theta>0$ determining accuracy of estimation.
1. Initialize $V(s)=-\frac{M}{1-\gamma}$, for all $s\in\mathcal{S}$, where $M$ is the upper bound on the absolute value of immediate rewards.
1. **Loop**:
    1. $\Delta \leftarrow 0$.
    1. **Loop for each** $s\in\mathcal{S}$:
        1. $v\leftarrow V(s)$.
        1. $V(s) \leftarrow \max_a\sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr]$.
        1. $\Delta\leftarrow \max(\Delta, \lvert v - V(s) \rvert)$.
1. **until** $\Delta < \theta$.
1. Output a deterministic policy, $\pi\approx\pi_*$, such that
$$ \pi(s) = \text{argmax}_a \sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr].$$

#### ACTS 1-2: Computing the sum & argmax of Value-Iteration. 

Compute the inner sum $$\sum_{s'} p(s'\mid s, a)\bigl[r(a\mid s, s') + \gamma V(s')\bigr],$$ given $V, s, a, \gamma$.

In [ ]:
# ACT 1: compute the inner sum of the equations above, given:
        # Value function V (an array of size num_states),
        # State s, Action a, and discount factor gamma.
        
def sum_over_states(V, s, a, gamma):
    sum_val = 0
    # recall that env.P is a list of tuples for (s,a): [(prob, s', r, done),...] (See code-block above for more details!)
    # where 'prob' is the transition probability P(s'|s,a). 
    # also, since r is associated with the states, you can think of the sum as only traversing states s'. 
    # therefore, all is needed is to iterate over each possible transition to state s'
    # from the pair (s,a), and compute the sum value (as in the pseudo-code).
    temp_prob = env.P[s][a]
    for i in range(len(temp_prob)): 
      sum_val += temp_prob[i][0] * (temp_prob[i][2] + gamma * V(temp_prob[i][1]))
    return sum_val


We now compute the argmax operation, in which we will update $\pi(a|s)$ where $s$ is the given state, and $a$ is the maximizing action! To obtain it, you should use the function above `sum_over_states`. 

In [ ]:
# ACT 2: update the action which can take us to a higher valued state, given:
        # Value function V (an array of size num_states),
        # Policy array pi (a matrix of shape [num_states, num_actions]),
        # State s, and discount factor gamma.
        
def argmax_over_actions(V, pi, s, gamma):
    max_val = 0
    temp_arr = []
    min_index = 0
    for a in range(num_actions): 
      temp_arr.append(sum_over_states(V,s,a,gamma))
    max_val = max(temp_arr)
    return pi, max_val

#### ACTS 3: Computing the Bellman update

We now perform a single iteration of update to the value function $V$, given state $s$, and discount factor $\gamma$. 
You may call a function you have implemented above, and output the difference between the new, updated value for this state, and the previous value (denoted as `delta` in the pseudo-code above).

In [ ]:
# ACT 3: update the Value function for state s, that is: V[s], by taking action which maximizes current value. Given:
        # Value function V (an array of size num_states),
        # State s, and discount factor gamma.

def bellman_optimality_update(V, s, gamma):  
    pi = np.zeros((num_states, num_actions))     
    v = V[s]
    pi, max_val = argmax_over_actions(V, pi, s, gamma)
    action_max = np.argmax(pi[s])
    V[s] = max_val
    delta = np.abs(v - V[s])
        # ACT3a: call a function implemented above, to get the action which maximizes current value.
        # ACT3b: update value V[s]
        # ACT 3c: compute and output delta.
    return delta

#### ACT 4: Initialize V
Initialize $V = -\frac{M}{1-\gamma}$, where $M$ is the upper bound on the absolute value of immediate rewards to be filled in.

In [ ]:
# ACT 4: Initialize V
# length_V gives the size of the vector V
def init_V(gamma, M, length_V):
    ### YOUR CODE GOES HERE
    V = np.ones(length_V) * (-1 * M/(1 - gamma))
    return V

#### ACT 5-6: Value-iteration (putting it all together)

In [ ]:
# Now, let's put it all together. Recall that we are given:
        # discount factor gamma, and wanted precision theta.
def value_iteration(gamma, theta):
    # Initialize V with init_V function
    ### YOUR CODE GOES HERE
    V = init_V(gamma, 1, num_states)
    # ACT 5: constrct the main loop, 
    #        iteratively calling the bellman update,
    #        and break when sufficient accuracy was reached. 
    while True: 
        delta = 0
        for s in range(num_states): 
          prev_delta = delta
          delta = max(prev_delta, bellman_optimality_update(V,s,gamma))
        if(delta < theta): 
          break
    pi = np.zeros((num_states, num_actions))

    pi = np.zeros((num_states, num_actions)) 
    # ACT 6: Extract optimal policy
    for s in range(num_states): 
      pi, _ = argmax_over_actions(V, pi, s, gamma)
        
    # output optimal value funtion, optimal policy
    return V, pi                                  


This next function `pretty_print` is just used to get some info printed out during the run.

Nothing for you to do here... 

In [ ]:
def pretty_print(i_episode, t, done, reward, total_rewards):
    print("Trial=" + str(i_episode)), print("Step=" + str(t)), print('')
    env.render()
    if done:
        print('')
        if reward == 1:
            total_rewards += 1
            print(">> Success! got the goal")
        else:
            print(">> Failed! fell into a hole")
        time.sleep(3), clear_output(wait=True)
    else:
      time.sleep(.5), clear_output(wait=True)
    it = max(i_episode, 1)
    print("Avg reward so far = %.2f" % (total_rewards / it))
    print('')
    return total_rewards


#### ACT 7: Call agent, and evaluate results

Below is the main loop of our setting: first, we call the `value_iteration` function, and obtain the optimal policy. 
Then, we run in many episodes (or trials) of the algorithm. In each trial, we start at the starting state "S". We then ask the agent what action to go towards. We make a step in the frozen lake with the chosen action (but may not move there because the lake is slippery!), our next state is given after we call `env.step(action)`. 

In [ ]:
# note that you can first try it out with a random agent, by setting this variable to True. 
random_agent = True
episodes = 100

if not random_agent:
    # We have called the value_iteration function:
    V_, pi = value_iteration(gamma, theta)
    # note that we don't need the value function V anymore, as we already extracted the optimal policy!

total_rewards = 0
for i_episode in range(episodes):
      state = env.reset()
      t = 0
      while True:
          t+=1
          # your agent picks an action: 
          if random_agent:
            action = env.action_space.sample() # totally random action !
          else:
            # ACT 7: get the best action according to optimal policy `pi`, and current state `state`.
            action = 0
            action = np.argmax(pi[state])
              
          # make a step according to policy
          state, reward, done, info = env.step(action) 
          # the above variables are:
                # state (object): agent's observation, current state (new state we've reached after the step we took)
                # reward (float) : amount of reward returned after previous action
                # done (bool): whether the episode has ended (only True if the state is the Goal or a Hole!)
                # info (dict): contains auxiliary diagnostic information (might be helpful for debugging, though not used in this assignment)

          # visualization
          total_rewards = pretty_print(i_episode, t, done, reward, total_rewards)
          if done: break

env.close()

### Conceptual Questions (ACTs 8-10)

Notice that in your implementation above, an agent that follows the value-iteration policy performs much better than an agent that follows the random policy (you can run both variations by setting the `random_agent` variable to `True/False`. 

Please **briefly** answer the following questions (1-2 sentences per question suffices):

#### Q1: Does the value-iteration policy *always* obtains reward=1? Explain why/why not.
#### <span style='color:cyan'>  Your answer here:It does not always obtain that award. Since it still has a 1/3 probability of going the wrong way. like if the right direction is is up, still 1/3 they go down or right!  </span>

#### Q2: Will the random policy *always* obtain reward=0? Explain why/why not.
#### <span style='color:cyan'>  Your answer here: again not "always" as there is always a possiblity that the penguin reaches the right state meaning thet obtained a reward of 1, however, random it is does not matter so long as at least once they reach the desired state. </span> 

#### Q3: When averaged over many episodes, will the value-iteration policy always obtain a larger reward than the random policy? Explain why/why not (no need to prove it, just state the appropriate claim shown in lecture and breifly explain how it implies this statement). 
#### <span style='color:cyan'>  Your answer here:It will always have a bigger value since the iterations are giving the max value of discount/reward. While the accuracy may increase over the trails since the value iteration policy starts bigger and grows with iterations and version it will always stay bigger!  </span>